# Vertailu: Reinforced learning vs Dynamic programming

Dynaamisella ohjelmoinnilla voi ratkaista elinkaarimalleja. Tässä verrataan dynaamisella ohjelmoinnilla ja RL:llä saatavia ratkaisuja yksinkertaisessa mallissa.

Tarkastellussa elinkaarimallissa _unemployment-v0_ on huomioitu vain kolme tilaa: työssä, työtön ja vanhuuseläkkeellä. Jokainen henkilö tekee vuosittain päätöksen työhönosallistumisesta ja alimman vanhuuseläkeiän täyttämisen jälkeen valinnan työn, työttömyyden ja vanhuuseläkkeen välillä. 

Mallissa palkat pidetään samoina läpi koko elinkaaren. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from lifecycle_rl import Lifecycle, DynProgLifecycle

%matplotlib inline
%pylab inline

# varoitukset piiloon (Stable baseline ei ole vielä Tensorflow 2.0-yhteensopiva, ja Tensorflow 1.5 valittaa paljon)
import warnings
warnings.filterwarnings('ignore')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Populating the interactive namespace from numpy and matplotlib


# Dynaaminen ohjelmointi

Ajetaan elämänkaarimallia sekä dynaamisella ohjelmoinnilla. Verrataan tuloksia, jotta näemme, miten hyvin RL toimii. Ajoajat eivät ole kovin vertailukelpoisia.

Dynaaminen ohjelmointi-koodi toimii ainoastaan minimaalisen mallin kanssa.

In [ ]:
cc3=DynProgLifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
cc3.train()
cc3.simulate()
cc3.plot_V(1)
cc3.render()

minimal model
Optimizing behavior


In [ ]:
cc3=DynProgLifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
cc3.simulate(save='results/dp_minimal',pop=2_000)
cc3.plot_actV(1)
cc3.render()

In [ ]:
cc3.plot_actV(40,emp=0,time_in_state=1)
cc3.plot_actV_diff(40)

Deterministinen versio, jossa tilasiirtymät eivät ole stokastisia

In [ ]:
cc6=Lifecycle(env='unemployment-v0',minimal=True,timestep=1.0,deterministic=False)
cc6.explain()
#cc6.simulate(pop=2_000,deterministic=False,load='saved/best_mini',rlmodel='acktr')
#cc6.train(train=True,steps=2_000_000,cont=True,rlmodel='acktr',save='saved/miniperus')
cc6.run_results(debug=False,steps1=2000,steps2=2_000_000,pop=2_000,deterministic=False,
                train=True,predict=True,batch1=1,batch2=10000,bestname='saved/best_mini',
                plot=True,save='saved/minimalli',cont=True,start_from='saved/best_mini',
                results='results/mini_simut_res',)
#cc6.simulate(pop=2_000,deterministic=False)


# Päätöksenteon vertailu

Lasketaan vertailukohta RL-menetelmällä ACKTR ja katsotaan paljonko tulokset eroavat.

In [ ]:
cc4.compare_with(cc3)
cc5.compare_with(cc3)
cc6.compare_with(cc3)

Tehdään sama RL-menetelmällä Deep Q-learning (dqn).

In [ ]:
cc5=Lifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
cc5.train(steps=5_000_000,cont=True,rlmodel='dqn',save='miniperus')
cc5.simulate(pop=2_000,rlmodel='dqn',load='miniperus',deterministic=True)

In [ ]:
cc7=Lifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
cc7.simulate(pop=2_000,deterministic=True,load='miniperus',rlmodel='dqn')

Deterministinen aktion valinta.

In [ ]:
cc6=Lifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
#cc6.train(steps=1_000_000,cont=False,rlmodel='acktr',save='miniperus')
cc6.simulate(pop=10_000,deterministic=True,load='miniperus',rlmodel='acktr',save='results/acktr_tulokset_miniperus_det')
#cc7=Lifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
#cc6.train(steps=1_000_000,cont=False,rlmodel='acktr',save='miniperus')
#cc7.simulate(pop=2_000,deterministic=True,load='miniperus',rlmodel='dqn')

In [ ]:
cc0=Lifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
cc0.load_sim('results/acktr_tulokset_miniperus')
cc1=Lifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
cc1.load_sim('results/acktr_tulokset_miniperus_det')
cc1.render()
cc2=Lifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
cc2.load_sim('results/dp_minimal')
cc2.render()

cc1.compare_with(cc0)
cc1.compare_with(cc2)
#cc5.compare_with(cc7)

In [ ]:
cc2.render()
cc1.render()
cc0.render()

# Päätöksenteon vertailu

Vertaillaan eri mallien tuloksia ja niiden eroja.

In [ ]:
cc4.compare_with(cc3)
cc5.compare_with(cc3)

In [ ]:
cc=DynProgLifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
ika=55
cc.plot_actV(ika,emp=1,time_in_state=0)
cc.RL_simulate_V(ika,rlmodel='acktr',emp=1,time_in_state=0,load='miniperus')
cc.RL_simulate_V(ika,rlmodel='acktr',emp=0,time_in_state=1,load='miniperus')
cc.RL_simulate_V(ika,rlmodel='acktr',emp=2,time_in_state=0,load='miniperus')

# Jakaumat

In [ ]:
cc=Lifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
cc.run_results(debug=False,steps1=500_000,steps2=100_000,n=100,pop=5_000,deterministic=True)
#cc.run_results(debug=False,steps1=10_000,steps2=10_000,n=10,train=False,predict=False)

In [ ]:
cc=Lifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
cc.run_results(debug=False,steps1=10_000_000,steps2=100_000,n=100,pop=5_000,deterministic=False)

In [ ]:
cc=Lifecycle(env='unemployment-v0',minimal=True,timestep=1.0)
cc.plot_simstats('results/simut_res_stats')